In [1]:
import requests
import lxml.html as lh
import pandas as pd
import re 

def getNationalTeamTable(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tables = doc.xpath('//table')
    for table in tables:
        captions = table.xpath('//caption')
        for caption in captions:
            text = caption.text_content()
            if "Appearances and goals by national team and year" in text:
                return table

def getAppsForPlayer(wiki_name):
    url = 'https://en.wikipedia.org/wiki/'+wiki_name
    table = getNationalTeamTable(url)
    if table:
        rows = table.xpath('//tr')
        i=0
        for row in rows:
            i=i+1
            if "Total" in row.text_content():
                if len(row) == 3:
                    total = row[1]
                    return int(total.text_content()), url
    return -1 ,url
    
def getPlayerRows(url):
    playerRows = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 12:
            name_cell = row[3]
            nationality_cell = row[2]
            text = name_cell.text_content().strip()
            skip = text.isnumeric() or len(text) == 0 or text == 'Name'
            if not skip:
                playerRows.append((name_cell,nationality_cell))
    return playerRows
    
    
def getWiki_Name(cell):
    links = list(cell.iterlinks())
    return links[0][2].replace('/wiki/','')
    
def get_country(cell):
    links = list(cell.iterlinks())
    return links[0][2].replace('/wiki/','')

def getWikiDetails(cells):
    player = cells[0]
    country_cell = cells[1]
    display_name = player.text_content()
    return display_name, getWiki_Name(player), get_country(country_cell)

def addPlayerApps(name_tuple):
    display_name = name_tuple[0]
    wiki_name = name_tuple[1]
    country = name_tuple[2]
    (apps,url) = getAppsForPlayer(wiki_name)
    return display_name, wiki_name, country, apps, url


def addPlayerApps(name_tuple):
    display_name = name_tuple[0]
    wiki_name = name_tuple[1]
    country = name_tuple[2]
    (apps,url) = getAppsForPlayer(wiki_name)
    return display_name, wiki_name, country, apps, url

def getAppearanceArray(url):
    playerRows = getPlayerRows(url)
    linksArray = [getWikiDetails(player) for player in playerRows]
    appsArray = [addPlayerApps(names) for names in linksArray]
    return appsArray



In [2]:
url = 'https://en.wikipedia.org/wiki/2019%E2%80%9320_Arsenal_F.C._season'
array = getAppearanceArray(url)
print(array)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.


[('Bernd Leno', 'Bernd_Leno', 'Germany', 8, 'https://en.wikipedia.org/wiki/Bernd_Leno'), ('Héctor Bellerín', 'H%C3%A9ctor_Beller%C3%ADn', 'Spain', 4, 'https://en.wikipedia.org/wiki/H%C3%A9ctor_Beller%C3%ADn'), ('Kieran Tierney', 'Kieran_Tierney', 'Scotland', -1, 'https://en.wikipedia.org/wiki/Kieran_Tierney'), ('Mohamed Elneny', 'Mohamed_Elneny', 'Egypt', 79, 'https://en.wikipedia.org/wiki/Mohamed_Elneny'), ('Sokratis Papastathopoulos', 'Sokratis_Papastathopoulos', 'Greece', 90, 'https://en.wikipedia.org/wiki/Sokratis_Papastathopoulos'), ('Henrikh Mkhitaryan', 'Henrikh_Mkhitaryan', 'Armenia', 88, 'https://en.wikipedia.org/wiki/Henrikh_Mkhitaryan'), ('Dani Ceballos', 'Dani_Ceballos', 'Spain', 11, 'https://en.wikipedia.org/wiki/Dani_Ceballos'), ('Alexandre Lacazette', 'Alexandre_Lacazette', 'France', 16, 'https://en.wikipedia.org/wiki/Alexandre_Lacazette'), ('Mesut Özil', 'Mesut_%C3%96zil', 'Germany', 92, 'https://en.wikipedia.org/wiki/Mesut_%C3%96zil'), ('Lucas Torreira', 'Lucas_Torreir